In [1]:
import os
import tensorflow as tf
from tensorflow import keras
os.environ['TF_CPP_MIN_LOG_LEVEL'] = "3"
import numpy as np
import json
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix   #copy

data = np.load('nn.npz')       #mark final.npz
label = data['y']
Bigdata = data['X']
Bigdata[np.isnan(Bigdata)] = 0.
# Bigdata = Bigdata[:,:,1:]
results = {}
results['acc'] = []
results['auc'] = []
results['confMat'] = []  #add
fold = 0
label[label < 5] = 0
label[label==5] = 1
y = label
index=np.arange(Bigdata.shape[0])
np.random.shuffle(index)
y=y[index]
Bigdata=Bigdata[index]
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=22)
for train, test in kfold.split(np.zeros(len(y)), y):
    print("run fold:{0}".format(fold))
    train_x, train_y = Bigdata[train], y[train]
    test_x, test_y = Bigdata[test], y[test]
    #train_y, test_y = tf.one_hot(train_y, depth=6).numpy(), tf.one_hot(test_y, depth=6).numpy()
    model = tf.keras.Sequential([
    keras.layers.Masking(mask_value=0., input_shape=(500, 7)), 
    keras.layers.Bidirectional(keras.layers.LSTM(256, return_sequences=True), input_shape=(500, 7)),
    keras.layers.Bidirectional(keras.layers.LSTM(128)),
    keras.layers.Dense(units=64, activation='relu'),
    keras.layers.Dense(units=1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss=keras.losses.BinaryCrossentropy(from_logits=False), metrics=['accuracy','AUC'])
    TensorBoard_callback = keras.callbacks.TensorBoard('logs_without_timestamp/' + str(fold) +"/" , histogram_freq=1)
    EarlyStopping_callback = keras.callbacks.EarlyStopping(patience=5)
    checkpoint_callback = keras.callbacks.ModelCheckpoint('checkpoint_without_timestamp/fold_' + str(fold) +"/" , save_best_only=True, save_weights_only=True)
    
    model.fit(x=train_x, y=train_y, batch_size=64, epochs=30, validation_split=0.2, callbacks=[TensorBoard_callback, EarlyStopping_callback, checkpoint_callback], verbose=2)

    #1
    y_pred = model.predict(test_x)
    roc = roc_auc_score(test_y, y_pred)
    y_pred[y_pred >= 0.5 ] =1
    y_pred[y_pred < 0.5] = 0
    acc = accuracy_score(test_y, y_pred)
    confMat = confusion_matrix(test_y, y_pred).astype(str)
    #acc = accuracy_score(np.argmax(test_y, axis=-1), np.argmax(y_pred, axis=-1))
    print("In the fold {0}: acc {1}, roc {2}".format(fold, acc, roc))
    results['acc'].append(acc)
    results['auc'].append(roc)
    confMat = [list(A)for A in confMat]
    results['confMat'].append(confMat)
    fold +=1
    #1
with open('auc_acc_without_timestamp.txt','a') as f:
    f.write(json.dumps(results))

run fold:0
Train on 288 samples, validate on 73 samples
Epoch 1/30
288/288 - 27s - loss: 0.6059 - accuracy: 0.6701 - AUC: 0.7609 - val_loss: 0.3523 - val_accuracy: 0.9178 - val_AUC: 0.9648
Epoch 2/30
288/288 - 2s - loss: 0.3382 - accuracy: 0.8611 - AUC: 0.9518 - val_loss: 0.2684 - val_accuracy: 0.9178 - val_AUC: 0.9739
Epoch 3/30
288/288 - 2s - loss: 0.2263 - accuracy: 0.8993 - AUC: 0.9705 - val_loss: 0.1313 - val_accuracy: 0.9726 - val_AUC: 0.9871
Epoch 4/30
288/288 - 2s - loss: 0.0890 - accuracy: 0.9757 - AUC: 0.9949 - val_loss: 0.0920 - val_accuracy: 0.9726 - val_AUC: 0.9909
Epoch 5/30
288/288 - 1s - loss: 0.0724 - accuracy: 0.9792 - AUC: 0.9964 - val_loss: 0.1274 - val_accuracy: 0.9452 - val_AUC: 0.9932
Epoch 6/30
288/288 - 1s - loss: 0.0523 - accuracy: 0.9861 - AUC: 0.9979 - val_loss: 0.1752 - val_accuracy: 0.9315 - val_AUC: 0.9883
Epoch 7/30
288/288 - 1s - loss: 0.0410 - accuracy: 0.9826 - AUC: 0.9992 - val_loss: 0.1083 - val_accuracy: 0.9589 - val_AUC: 0.9955
Epoch 8/30
288/288 